In [1]:
import pandas as pd
import numpy as np
from mcts import MCTS
from farmgame import Farm
from farmgame import Transition

In [2]:
dataset = pd.read_csv('../data/subjgamedf.csv')

In [3]:
def row_to_state(row):
    # Convert a row of the dataset into a Farm state
    config = {
        "redplayer": {
            "name": "red",
            "loc": {"x": 2, "y": 15},  # Example fixed starting position
            "capacity": row["nRedVeg"],  # Use data to set player capacity
            "contents": [],  # Assume backpack starts empty
        },
        "purpleplayer": {
            "name": "purple",
            "loc": {"x": 3, "y": 16},  # Example fixed starting position
            "capacity": row["nPurpVeg"],  # Use data to set player capacity
            "contents": [],  # Assume backpack starts empty
        },
        "redfirst": row["redFirst"],  # Determine starting player from the data
        "items": row["objectLayer"],  # Assign items layer based on the game state
        "stepcost": 1 if row["costCond"] == "low" else 2,
        "pillowcost": 5,  # Fixed for now
        "condition": {
            "resourceCond": row["resourceCond"],
            "costCond": row["costCond"],
            "visibilityCond": row["visibilityCond"],
        },
    }
    return Farm(config)

In [4]:
def simulate_helping_behavior(row, policy="selfish"):
    # Initialize the game state
    # state = configure_game(...)
    state = row_to_state(row)
    mcts_agent = MCTS(time=5, policy=policy)
    mcts_agent.update(state)

    # Get the best action
    best_action = mcts_agent.choose_action()

    # Determine if the action involves helping
    transition = Transition(state, best_action)
    return transition.is_helping()


In [5]:
# Apply MCTS to the dataset
dataset["predicted_helping_event"] = dataset.apply(
    lambda row: simulate_helping_behavior(row, policy="selfish"), axis=1
)

AttributeError: 'str' object has no attribute 'id'

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(dataset["helping_event"], dataset["predicted_helping_event"])
print(f"Accuracy: {accuracy}")